In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Import Necessary Libraries

import os
import numpy as np
import pandas as pd
import logging
import warnings as warnings
import random
import pickle

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ReduceLROnPlateau,
    TensorBoard,
    ModelCheckpoint,
    Callback
)

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [2]:
# Configuration and Setup

# Suppress warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,  # Set to DEBUG level for detailed logging
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Enable memory growth for GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Enabled memory growth for {len(gpus)} GPU(s).")
    except RuntimeError as e:
        print(f"Error setting memory growth: {e}")


Enabled memory growth for 2 GPU(s).


In [3]:
# Define File Paths and Parameters

test_dir = '/kaggle/input/ai-knight/dataset/test'
train_dir = '/kaggle/input/ai-knight/dataset/train'

IMG_SIZE = (32,32)
BATCH_SIZE = 32


In [4]:
# Data Loading and Preprocessing

# Load data
# Load training data
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,            # Reserve 20% of data for validation
    subset="training",               # Load the training subset
    seed=42,                         # Set seed for reproducibility
    labels="inferred",               # Automatically assigns labels
    label_mode="int",                # Outputs integer labels
    image_size=IMG_SIZE,             # Resize images to target size
    batch_size=BATCH_SIZE,           # Batch size
    shuffle=True                     # Shuffle the data
)

#load val data '
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,            # Reserve 20% of data for validation
    subset="validation",             # Load the validation subset
    seed=42,                         # Set seed for reproducibility
    labels="inferred",               # Automatically assigns labels
    label_mode="int",                # Outputs integer labels
    image_size=IMG_SIZE,             # Resize images to target size
    batch_size=BATCH_SIZE,           # Batch size
    shuffle=True                     # Shuffle the data
)
#fake - 0, real - 1

Found 100000 files belonging to 2 classes.
Using 80000 files for training.
Found 100000 files belonging to 2 classes.
Using 20000 files for validation.


In [5]:
print(f"Training batches: {len(train_dataset)}")
print(f"Validation batches: {len(val_dataset)}")
print(train_dataset.class_names)  # Output: ['fake', 'real']

Training batches: 2500
Validation batches: 625
['fake', 'real']


In [6]:
#build the model 

input_shape = IMG_SIZE + (3,)
base_model = EfficientNetB3(include_top=False, weights='imagenet', input_shape=input_shape)
base_model.trainable = False  # Freeze the base model
# print("Loaded EfficientNetB3 base model with ImageNet weights.")

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

# Add the output layer
output = Dense(1, activation='sigmoid')(x)  # Single unit with sigmoid activation for binary classification

# Define the complete model
model = Model(inputs=base_model.input, outputs=output)
print("Model architecture created.")

# Print model summary
model.summary()

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Model architecture created.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 32, 32, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 32, 32, 3)      │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 32, 32, 3)      │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 32, 32, 3)      │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 33, 33, 3)      │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 16, 16, 40)     │          1,080 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 16, 16, 40)     │            160 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 16, 16, 40)     │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 16, 16, 40)     │            360 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 16, 16, 40)     │            160 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 16, 16, 40)     │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 40)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 40)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 10)       │            410 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 10,791,216 (41.17 MB)

 Trainable params: 4,609 (18.00 KB)

 Non-trainable params: 10,786,607 (41.15 MB)

In [7]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  # Use Adam optimizer with a learning rate of 1e-3 
    loss='binary_crossentropy',                              # Binary cross-entropy loss for binary classification
    metrics=['accuracy']                                     # Track accuracy during training
)

In [8]:
# Define Callbacks

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',                # Monitor validation loss
    patience=5,                        # Stop training if no improvement for 5 epochs
    restore_best_weights=True,         # Restore model weights from the epoch with the best value
    verbose=1                          # Print logs when stopping
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',                # Monitor validation loss
    factor=0.5,                        # Reduce learning rate by half
    patience=2,                        # Wait for 2 epochs before reducing
    verbose=1                          # Print logs about learning rate changes
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.keras',       # Filepath to save the best model
    monitor='val_loss',                # Monitor validation loss
    save_best_only=True,               # Save only if the model improves
    verbose=1                          # Print logs when saving
)

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='./logs',                  # Directory to save logs
    histogram_freq=1                   # Frequency (in epochs) to compute histograms
)

class BatchLoggingCallback(tf.keras.callbacks.Callback): #can be removed 
    def __init__(self, log_every=10):
        super(BatchLoggingCallback, self).__init__()
        self.log_every = log_every

    def on_train_batch_end(self, batch, logs=None):
        if (batch + 1) % self.log_every == 0:
            log_message = f"Batch {batch + 1}: " + ", ".join([f"{k}={v:.4f}" for k, v in logs.items()])
            print(log_message)

batch_logging = BatchLoggingCallback(log_every=10)



callbacks = [early_stop, reduce_lr, checkpoint, tensorboard_callback, batch_logging]


In [9]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,                      # Set the maximum number of epochs
    callbacks=callbacks,             # Add callbacks to monitor and enhance training
    verbose=1
    #also give step per epoch
)

Epoch 1/50
   9/2500 ━━━━━━━━━━━━━━━━━━━━ 1:03 25ms/step - accuracy: 0.5017 - loss: 0.8698Batch 10: accuracy=0.5375, loss=0.8098
  19/2500 ━━━━━━━━━━━━━━━━━━━━ 1:13 29ms/step - accuracy: 0.5349 - loss: 0.8280Batch 20: accuracy=0.5875, loss=0.7568
  29/2500 ━━━━━━━━━━━━━━━━━━━━ 1:16 31ms/step - accuracy: 0.5568 - loss: 0.7962Batch 30: accuracy=0.6115, loss=0.7170
  39/2500 ━━━━━━━━━━━━━━━━━━━━ 1:18 32ms/step - accuracy: 0.5718 - loss: 0.7742Batch 40: accuracy=0.6250, loss=0.6939
  49/2500 ━━━━━━━━━━━━━━━━━━━━ 1:18 32ms/step - accuracy: 0.5839 - loss: 0.7574Batch 50: accuracy=0.6394, loss=0.6871
  59/2500 ━━━━━━━━━━━━━━━━━━━━ 1:17 32ms/step - accuracy: 0.5940 - loss: 0.7460Batch 60: accuracy=0.6469, loss=0.6885
  69/2500 ━━━━━━━━━━━━━━━━━━━━ 1:17 32ms/step - accuracy: 0.6020 - loss: 0.7370Batch 70: accuracy=0.6522, loss=0.6781
  79/2500 ━━━━━━━━━━━━━━━━━━━━ 1:16 32ms/step - accuracy: 0.6087 - loss: 0.7295Batch 80: accuracy=0.6594, loss=0.6753
  89/2500 ━━━━━━━━━━━━━━━━━━━━ 1:16 32ms/step

In [10]:
#Fine tuuning model start 


# Unfreeze the base model
base_model.trainable = True

# Freeze the first 100 layers (optional, based on your use case)
for layer in base_model.layers[:100]:
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),  # Reduced learning rate
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

# Fine-tuning training
fine_tune_epochs = 10  # Adjust based on experiment results

history_fine = model.fit(
    train_dataset,                    # Your training dataset
    validation_data=val_dataset,      # Your validation dataset
    epochs=fine_tune_epochs,          # Number of fine-tuning epochs
    callbacks=callbacks,              # Previously defined callbacks
    verbose=1                         # Displays training progress
)


Epoch 1/10
   9/2500 ━━━━━━━━━━━━━━━━━━━━ 1:20 32ms/step - accuracy: 0.4544 - loss: 1.0885Batch 10: accuracy=0.4531, loss=1.1002
  17/2500 ━━━━━━━━━━━━━━━━━━━━ 1:13 30ms/step - accuracy: 0.4574 - loss: 1.0792Batch 20: accuracy=0.4437, loss=1.0561
  29/2500 ━━━━━━━━━━━━━━━━━━━━ 1:22 33ms/step - accuracy: 0.4531 - loss: 1.0660Batch 30: accuracy=0.4448, loss=1.0335
  39/2500 ━━━━━━━━━━━━━━━━━━━━ 1:28 36ms/step - accuracy: 0.4534 - loss: 1.0532Batch 40: accuracy=0.4625, loss=0.9994
  49/2500 ━━━━━━━━━━━━━━━━━━━━ 1:31 37ms/step - accuracy: 0.4558 - loss: 1.0398Batch 50: accuracy=0.4656, loss=0.9796
  57/2500 ━━━━━━━━━━━━━━━━━━━━ 1:28 36ms/step - accuracy: 0.4579 - loss: 1.0314Batch 60: accuracy=0.4781, loss=0.9677
  68/2500 ━━━━━━━━━━━━━━━━━━━━ 1:24 35ms/step - accuracy: 0.4612 - loss: 1.0206Batch 70: accuracy=0.4772, loss=0.9515
  79/2500 ━━━━━━━━━━━━━━━━━━━━ 1:24 35ms/step - accuracy: 0.4631 - loss: 1.0110Batch 80: accuracy=0.4730, loss=0.9509
  89/2500 ━━━━━━━━━━━━━━━━━━━━ 1:26 36ms/step

In [13]:
# Save the final trained model
final_model_path = '/kaggle/working/final_model.h5'
model.save(final_model_path)
print(f"Final model saved as {final_model_path}.")

Final model saved as /kaggle/working/final_model.h5.


In [15]:
from tensorflow.keras.models import load_model
model = load_model(final_model_path)
print("Model loaded successfully.")

Model loaded successfully.


In [16]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels=None,                     # Test data may not have labels
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False                    # Keep test data in order for evaluation
)


Found 500 files.


In [19]:
test_file_paths = test_dataset.file_paths  # Use file_paths to get the paths of the test images

# Predict on the test dataset
predictions = model.predict(test_dataset)
predicted_classes = (predictions > 0.5).astype(int).flatten()  # Convert probabilities to binary classes

# Create a DataFrame for the predictions
output_df = pd.DataFrame({
    'ImageID': [os.path.basename(f) for f in test_file_paths],  # Extract filenames only
    'Prediction': predicted_classes  # Predictions (0 or 1)
})

# Save the predictions to a CSV file
output_csv_path = '/kaggle/working/test_predictions.csv'
output_df.to_csv(output_csv_path, index=False)
print(f"Predictions saved to {output_csv_path}.")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Predictions saved to /kaggle/working/test_predictions.csv.
